In [ ]:
import gdown
!mkdir data

gdown.download('https://drive.google.com/uc?id=1k4_wqMaAApry4gkHHyyad1oH3ZXyDf2j&export=download', 'data.7z',quiet=False)

!7z x data.7z -odata

In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
import os
import dlib
import glob

import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

import numpy as np
%matplotlib inline

In [ ]:
img_black = np.zeros((70,140,3))

base_dir = 'data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

classes = 'Begin, Choose, Connection, Navigation, Next, Previous, Start, Stop, Hello, Web'
classes = classes.split(', ')

predictor_path = 'assests/predictors/shape_predictor_68_face_landmarks.dat'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

train_people = 'F01, F02, F04, F05, F06, F07, F08, F09, M01, M02, M04'.split(', ')
val_people = 'F10, M07'.split(', ')
test_people = 'F11, M08'.split(', ')
print(train_people)
print(val_people)
print(test_people)

classes_num = ['0'+str(i) if i < 10 else str(i) for i in range(1, 11) ]
word_ids = ['0'+str(i) if i < 10 else str(i) for i in range(1, 11) ]
classes_dict = dict(zip(classes_num, classes))
print(classes_num)
print(word_ids)
print(classes_dict)

In [ ]:
def show_img(img, figsize=(8, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.grid(False)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.imshow(img)
    plt.imshow(img)

In [ ]:
train_image = []
train_class = []

for class_id in tqdm(range(len(classes))):
    images = sorted(glob.glob('data/train/' + classes[class_id] + '/*.jpg'))
    for i in range(len(images)):
        if ('noised' in images[i] or 'rand_contr' in images[i]
        or 'vert_flip' in images[i] or 'hor_flip' in images[i]):
            train_image.append(images[i].split('/')[3])
            train_class.append(classes_dict[images[i].split('/')[3].split(']')[-1][:2]])
        else:
            train_image.append(images[i].split('/')[3])
            train_class.append(classes_dict[images[i].split('/')[3].split('_')[1]])
        
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

train_data.to_csv('data/train_new.csv',header=True, index=False)


val_image = []
val_class = []

for class_id in tqdm(range(len(classes))):
    images = sorted(glob.glob('data/validation/' + classes[class_id] + '/*.jpg'))
    for i in range(len(images)):
        val_image.append(images[i].split('/')[3])
        val_class.append(classes_dict[images[i].split('/')[3].split('_')[0]])
val_data = pd.DataFrame()
val_data['image'] = val_image
val_data['class'] = val_class

val_data.to_csv('data/val_new.csv',header=True, index=False)

In [ ]:
train = pd.read_csv('data/train_new.csv')
val = pd.read_csv('data/val_new.csv')

y_tr = train['class']
y_tr_dummy = pd.get_dummies(y_tr)

y_val = val['class']
y_val_dummy = pd.get_dummies(y_val)

In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
val.head()

In [ ]:
val.tail()

In [ ]:
print(len(train))
print(len(val))

In [ ]:
import keras

In [ ]:
!wget https://gist.githubusercontent.com/Emadeldeen-24/736c33ac2af0c00cc48810ad62e1f54a/raw/f2428c9ba7c13e08e2aa9b2d5ff64ad575a99dd8/tweaked_ImageGenerator_v2.py

In [ ]:
from tweaked_ImageGenerator_v2 import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator()
train_data = datagen.flow_from_directory('data/train', target_size=(70, 140), batch_size=3, frames_per_step=12, shuffle=False)

In [ ]:
val_data = datagen.flow_from_directory('data/validation', target_size=(70, 140), batch_size=3, frames_per_step=12, shuffle=False)

In [ ]:
import math

print(math.ceil(46872/(3*12)))
print(math.ceil((1510/(3*12))))

In [ ]:
from keras.models import Model

In [ ]:
video = Input(shape=(22, 50, 100, 3))

cnn_base = MobileNetV2(input_shape=(50, 100, 3), include_top=False)
cnn_out = GlobalAveragePooling2D()(cnn_base.output)

cnn = Model(input=cnn_base.input, output=cnn_out)
encoded_frames = TimeDistributed(cnn)(video)

encoded_sequence = LSTM(54)(encoded_frames)
outputs = Dense(10, activation="softmax")(encoded_sequence)

model = Model([video], outputs)

In [ ]:
adam2 = optimizers.Adam(lr=0.0001)
save_weights = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model.compile(loss='categorical_crossentropy',optimizer=adam2,metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit_generator(train_data, epochs=50, steps_per_epoch=1302, 
                              validation_data=val_data, validation_steps=42, shuffle=True)